In [1]:
a = 2
b = 3
a+b

5

In [19]:


# """
# Imports necessary packages and creates a function (fred_csv()) which scrapes data from the webpage as a csv file and adds it to a Data Frame

# | Indicator                 | FRED Series ID |
# | ------------------------- | -------------- |
# | Unemployment Rate         | `UNRATE`       |
# | Job Openings Rate         | `JTSJOR`       |
# | Quits Rate                | `JTSQUR`       |
# | Layoffs & Discharges Rate | `JTSLDR`       |
# | Initial Claims (weekly)   | `ICSA`         |
# | Labor Force Participation | `CIVPART`      |

# """
import pandas as pd
import numpy as np
import requests

# Your FRED API key
FRED_API_KEY = "db6b2f598b3e77cf4064ccb8cde7fa37"

def fred(series_id, start, end):
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "observation_start": start,
        "observation_end": end,
    }

    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()

    obs = r.json()["observations"]

    s = pd.Series(
        [o["value"] for o in obs],
        index=pd.to_datetime([o["date"] for o in obs]),
        name=series_id
    )

    s = pd.to_numeric(s.replace(".", np.nan), errors="coerce")
    return s


def fred_bundle(series_ids, start, end, weekly_to_monthly=None):
    weekly_to_monthly = set(weekly_to_monthly or [])
    series_list = []

    for sid in series_ids:
        s = fred(sid, start, end)

        # Convert weekly to monthly if needed
        if sid in weekly_to_monthly:
            s = s.resample("ME").mean()
        else:
            s = s.resample("ME").last()

        series_list.append(s)

    df = pd.concat(series_list, axis=1).dropna()
    return df


# -----------------------------------
# Pull your labor market data
# -----------------------------------

start = "2001-01-01"
end   = "2026-12-31"   # safe to set ahead

series_ids = ["UNRATE", "JTSJOR", "JTSQUR", "JTSLDR", "ICSA", "CIVPART"]

data = fred_bundle(series_ids, start, end, weekly_to_monthly=["ICSA"])

print("Latest month with all series:", data.index.max())
data.tail()


Latest month with all series: 2025-12-31 00:00:00


,UNRATE,JTSJOR,JTSQUR,JTSLDR,ICSA,CIVPART
2025-07-31,4.3,4.3,2.0,1.1,221250.0,62.2
2025-08-31,4.3,4.3,1.9,1.1,230000.0,62.3
2025-09-30,4.4,4.6,2.0,1.1,234750.0,62.5
2025-11-30,4.5,4.2,2.0,1.1,217600.0,62.5
2025-12-31,4.4,3.9,2.0,1.1,219000.0,62.4
